In [1]:
from sympy.interactive import printing
printing.init_printing(use_latex=True)
from sympy import Eq, solve_linear_system, Matrix, Function
from copy import deepcopy
import numpy as np
import sympy as sp

"""
    A compilation of the theory that I learned during my NCS course.
    This is not a homework. I made this for my own training and understanding.
    Due to limited time and effort. Some exercises has been opted out.
"""

'\n    A compilation of the theory that I learned during my NCS course.\n    This is not a homework. I made this for my own training and understanding.\n    Due to limited time and effort. Some exercises has been opted out.\n'

In [2]:
def get_math_operator(s:str, inverse=False):
    """ Need more operators! """
    if inverse:
        s = s.replace('+', '-').replace('*', '/')
    arr = []
    s = s.replace(" ", "")
    for i in range(len(s)):
        if s[i] in ['*', '+', '/', '-']:
            arr.append(s[i])
    return arr

def evaluate_op(x, y, op: str):
    if op == '*':
        x *= y
    elif op == '+':
        x += y
    elif op == '-':
        x -= y
    elif op == '/':
        x /= y
    return x

def sort_states(states):
    n = len(states)
    st = [0 for i in range(n)]
    ind = np.array([int(str(i).replace(str(i)[0],"")) for i in states])
    if max(ind)>len(ind):
        ind -= (max(ind)-len(ind))
    for i, val in enumerate(states):
        st[ind[i]-1] = val
    return st

def diff_t(f, expressions):
    f.free_symbols  # Get variables
    df = ""
    for i in f.free_symbols:
        deriv = str(sp.diff(f, i)) + '*'
        if '1.0*' in deriv:
            deriv = deriv.replace('1.0*', '')
        elif '1*' in deriv:
            deriv = deriv.replace('1*', '')
        i_d = deriv + str(i) + 'd'
        df += i_d+"+"
    df = df[0:-1]  # Remove excess operator
    
    ## Get operators and variables from String
    Vd_operator = get_math_operator(df)
    Vd_variable = df.replace('+', '*').replace('-', '*').split('*')
    
    ## Get math operator priority
    hp = []  # High priority
    lp = []
    for i, val in enumerate(Vd_operator):
        if val in ['*', '/']:
            hp.append(i)
        elif val in ['+', '-']:
            lp.append(i)

    ## Change df from str to sym
    if lp == []:
        Vd_new = [expressions[str(df)]]
    else:
        # Get multiplication/division syms
        Vd_multi = []
        Vd_new = []
        i = 0
        for i in hp:
            var1 = expressions[Vd_variable[i]]
            var2 = expressions[Vd_variable[i+1]]
            Vd_multi.append(evaluate_op(var1, var2, Vd_operator[i]))
        # Add/substract syms
        for i, val in enumerate(lp):
            if Vd_multi == []:
                var1 = expressions[Vd_variable[i]]
                var2 = expressions[Vd_variable[i+1]]
            else:
                var1 = Vd_multi[i]
                var2 = Vd_multi[i+1]
            Vd_new.append(evaluate_op(var1, var2, Vd_operator[val]))
            
    df = sp.factor(Vd_new[0])
    return df

def get_sym_dict(Xd):
    n = len(Xd)
    g = Matrix([Xd[i].coeff(u) for i in range(n)])
    f = Xd - g*u
    states = sort_states(f.free_symbols)
    sym_dict = {f"x{i+1}d":Xd[i] for i in range(len(Xd))}
    sym_dict.update({f"x{i+1}":val for i, val in enumerate(states)})
    return sym_dict

def evaluate_eigenvalue(w):
    print(f"Eigenvalues are: {w}")
    complex = (np.iscomplex(w)).all()
    if (np.real(w) < 0).all():
        if not complex:
            print("System is stable")
        else:
            print("System is stable focus")
    elif (np.real(w) > 0).all():
        print("System is unstable")
    elif (np.real(w) == 0).all():
        print("Eigenvalue is pure imaginary, stability analysis not possible for nonlinear systems!")
        
def get_relative_degree(A, y):
    n = len(A)
    g = Matrix([A[i].coeff(u) for i in range(n)])
    f = A - g*u
    states = sort_states(f.free_symbols)
    v = 1
    Lfh = y
    for i in range(n):
        LgLfh = g.T*Matrix([sp.diff(Lfh, i) for i in states])
        print(f"{v=} | LgLfh={LgLfh[0]} | Lfh={Lfh}")
        if not LgLfh[0]==0:
            break
        Lfh = f.T*Matrix([sp.diff(Lfh, i) for i in states])
        v += 1
    return v

def check_lyapunov_stability(V, Xd):
    # Derivate dV/dt
    # Substitute x in Vd
    # Check if negative definite
    # If semi negdef, do la salle
    return

def transform_and_check_stability(Xd, Z_dict):
    sym_dict = get_sym_dict(Xd)
    z1d = diff_t(Z_dict['z1'], sym_dict)
    z2d = diff_t(Z_dict['z2'], sym_dict)
    z3d = diff_t(Z_dict['z3'], sym_dict)

    for i in [x1, x2, x3]:
        z1d = z1d.subs(i, Z_dict[str(i)])
        z2d = z2d.subs(i, Z_dict[str(i)])
        z3d = z3d.subs(i, Z_dict[str(i)])

    # Find u
    eq1 = sp.Function('eq1')
    eq1 = Eq(z1d, v)
    Z_dict['u'] = sp.solve(eq1, u)[0]
    Z = Matrix([z2d, z3d])

    # Evaluate internal dynamics
    for i in range(len(Z)):
        Z[i] = Z[i].subs([(u, Z_dict['u']), (v, 0), (z1, 0)])

    display(Z)
    Y = sp.Matrix([i for i in sort_states(Z.free_symbols)])
    A = Z.jacobian(Y)
    for i in Y:
        A = A.subs(i, 0)
    display(A)
    A = np.array(A).astype(np.float64)
    try:
        w, _ = np.linalg.eig(A)
        evaluate_eigenvalue(w)
    except TypeError:
        print("Eigenvalue on the imaginary axis, stability analysis not possible!")
    return

def get_z_sym(a, b, c):
    # Change list of Z into expendable strings
    # Make array of string based on argument
    # For loop dictionary with sp.Symbols(str)
    Z = [z1, z2, z3]
    Z_dict = {'z1': a, 'z2': b, 'z3': c}
    Z_copy = deepcopy(Z_dict)
    for i, val in enumerate(Z_copy.items()):
        if len(str(val[1]))>3:
            Vd_operator = get_math_operator(str(val[1]), inverse=True)
            Vd_variable = str(val[1]).replace(" ", '').replace('+', '*').replace('-', '*').split('*')
            temp = (i, val)
        else:
            Z_dict.update({str(val[1]): Z[i]})
    try:
        if temp:
            Z_dict.update({str(Vd_variable[0]): evaluate_op(Z[temp[0]], Z_dict[Vd_variable[1]], Vd_operator[0])})
    except NameError:
        pass
    return Z_dict

# Backstepping
def backstepping_control():
    # find suitable x2 = theta(x1)
    # insert to x1d
    # calculate lyapunov stability
    # transform to z states
    # derive z with respect to t
    # substitute x variables to z
    # substitute z2 to u_
    # calculate next lyapunov stability
    # find suitable u_ to fulfill stability
    # rearrange to u
    # transform z to x states
    pass

In [4]:
x1, x2, x3, u = sp.symbols('x1 x2 x3 u')
z1, z2, z3, v = sp.symbols('z1 z2 z3 v')

In [37]:
x1d = x2 - x3
x2d = -x1*x3 - 2*x2 + u
x3d = x1 + u
Xd = Matrix([x1d, x2d, x3d])

In [38]:
# Exercise 1a
y = x1
print(f"Relative degree is {get_relative_degree(Xd, y)}")

v=1 | LgLfh=0 | Lfh=x1
v=2 | LgLfh=0 | Lfh=Matrix([[x2 - x3]])
v=3 | LgLfh=-x1 - 2 | Lfh=Matrix([[-x1*x3 - x1 - 2*x2]])
Relative degree is 3


In [39]:
# Exercise 1b
y = x2
print(f"Relative degree is {get_relative_degree(Xd, y)}")
Z = get_z_sym(x2, x1, x3)
print(f"{Z=}")
transform_and_check_stability(Xd, Z)

v=1 | LgLfh=1 | Lfh=x2
Relative degree is 1
Z={'z1': x2, 'z2': x1, 'z3': x3, 'x2': z1, 'x1': z2, 'x3': z3}


⎡   -z₃    ⎤
⎢          ⎥
⎣z₂⋅z₃ + z₂⎦

⎡0  -1⎤
⎢     ⎥
⎣1  0 ⎦

Eigenvalues are: [0.+1.j 0.-1.j]
Eigenvalue is pure imaginary, stability analysis not possible for nonlinear systems!


In [40]:
# Exercise 1c
y = x3
print(f"Relative degree is {get_relative_degree(Xd, y)}")
Z = get_z_sym(y, x1, x2)
print(f"{Z=}")
transform_and_check_stability(Xd, Z)

v=1 | LgLfh=1 | Lfh=x3
Relative degree is 1
Z={'z1': x3, 'z2': x1, 'z3': x2, 'x3': z1, 'x1': z2, 'x2': z3}


⎡    z₃    ⎤
⎢          ⎥
⎣-z₂ - 2⋅z₃⎦

⎡0   1 ⎤
⎢      ⎥
⎣-1  -2⎦

Eigenvalues are: [-1. -1.]
System is stable


In [41]:
# Exercise 1d
y = x2 + x3
print(f"Relative degree is {get_relative_degree(Xd, y)}")
Z = get_z_sym(y, x1, x3)
print(f"{Z=}")
transform_and_check_stability(Xd, Z)

v=1 | LgLfh=2 | Lfh=x2 + x3
Relative degree is 1
Z={'z1': x2 + x3, 'z2': x1, 'z3': x3, 'x1': z2, 'x3': z3, 'x2': z1 - z3}


⎡     -2⋅z₃     ⎤
⎢               ⎥
⎢z₂⋅z₃   z₂     ⎥
⎢───── + ── - z₃⎥
⎣  2     2      ⎦

⎡ 0   -2⎤
⎢       ⎥
⎣1/2  -1⎦

Eigenvalues are: [-0.5+0.8660254j -0.5-0.8660254j]
System is stable focus


In [42]:
# Exercise 2b
x1d = -x1 + 2*x2
x2d = x1**2 + x3 + u
x3d = -4*x3 + x2**2 + u
Xd = Matrix([x1d, x2d, x3d])

y = x1
print(f"Relative degree is {get_relative_degree(Xd, y)}")
# Can't work with these, need to change how to code works
# Z = get_z_sym(y, x1d, x3)
# Z = {'z1': x1, 'z2': x1d, 'z3': x3, 'x2': 0.5*(z1+z2), 'x3': z3, 'x1': z1}
# transform_and_check_stability(Xd, Z)

v=1 | LgLfh=0 | Lfh=x1
v=2 | LgLfh=2 | Lfh=Matrix([[-x1 + 2*x2]])
Relative degree is 2


In [43]:
# Lyapunov

# Exercise 1
x1d = -x1**3 + x1*x2**2
x2d = -x1**2*x2 - x2**3
Xd = sp.Matrix([x1d, x2d])
V = 1/2*(x1**2 + x2**2)
Vd = diff_t(V, get_sym_dict(Xd))
print(f"{Vd=}")

# Exercise 2
x1d = 2*x2 - x1
x2d = -2*x1 - x2
Xd = sp.Matrix([x1d, x2d])
V = 1/2*(x1**2 + x2**2)
Vd = diff_t(V, get_sym_dict(Xd))
print(f"{Vd=}")

# Exercise 3.3
x1d = -3*x1 - x1**3 - x1*x2
x2d = -2*x1**2 - x2 + x1**2
Xd = sp.Matrix([x1d, x2d])
V = 1/2*(x1**2 + x2**2)
Vd = diff_t(V, get_sym_dict(Xd))
print(f"{Vd=}")

# Exercise 3.4
x1d = -40*x1*x2**2 + 10*x2 - 10*x1**3
x2d = 40*x1**3 - 10*x1
Xd = sp.Matrix([x1d, x2d])
V = 1/2*(x1**2 + x2**2)
Vd = diff_t(V, get_sym_dict(Xd))
print(f"{Vd=}")

Vd=-x1**4 - x2**4
Vd=-x1**2 - x2**2
Vd=-x1**4 - 2*x1**2*x2 - 3*x1**2 - x2**2
Vd=-10*x1**2*(x1 - 2*x2)**2


In [62]:
# Sliding Mode

# Exercise 1
th = sp.symbols('th')
x1d = -1*x1**2 + x2 + u
x2d = x1 + x2
Xd = sp.Matrix([x1d, x2d])
display(Xd)

⎡      2     ⎤
⎢u - x₁  + x₂⎥
⎢            ⎥
⎣  x₁ + x₂   ⎦

In [55]:
# 1a
s1 = x1 - x2
v = get_relative_degree(Xd, s1)
if s1.subs([(x1, 0), (x2, 0)]) == 0:
    print("Condition 1 is fulfilled")
if v == 1:
    print(f"Condition 2 is fulfilled")
# Z = {'z1': s1, 'z2': x2, 'x1': z1+z2, 'x2': z2}
# transform_and_check_stability(Xd, Z)

s2 = x1 + 4*x2
v = get_relative_degree(Xd, s2)
if s1.subs([(x1, 0), (x2, 0)]) == 0:
    print("Condition 1 is fulfilled")
if v == 1:
    print(f"Condition 2 is fulfilled")
# Z = {'z1': s2, 'z2': x2, 'x1': z1-4*z2, 'x2': z2}

s3 = x2**2 + x2
v = get_relative_degree(Xd, s3)
if s1.subs([(x1, 0), (x2, 0)]) == 0:
    print("Condition 1 is fulfilled")
if v == 1:
    print(f"Condition 2 is fulfilled")
else:
    print(f"Condition 2 is NOT fulfilled")

v=1 | LgLfh=1 | Lfh=x1 - x2
Condition 1 is fulfilled
Condition 2 is fulfilled
v=1 | LgLfh=1 | Lfh=x1 + 4*x2
Condition 1 is fulfilled
Condition 2 is fulfilled
v=1 | LgLfh=0 | Lfh=x2**2 + x2
v=2 | LgLfh=2*x2 + 1 | Lfh=Matrix([[(x1 + x2)*(2*x2 + 1)]])
Condition 1 is fulfilled
Condition 2 is NOT fulfilled


In [59]:
# 1b
# Suitable Manifold: s(x) = x1 + 4*x2
s = x1 + 4*x2

⎡      2     ⎤
⎢u - x₁  + x₂⎥
⎢            ⎥
⎣  x₁ + x₂   ⎦

In [83]:
# Exercise 2
x1d = x1*x2
x2d = x1 + u
Xd = sp.Matrix([x1d, x2d])

s = x2+x1**2
v = get_relative_degree(Xd, s)
if s1.subs([(x1, 0), (x2, 0)]) == 0:
    print("Condition 1 is fulfilled")
if v == 1:
    print(f"Condition 2 is fulfilled")
else:
    print(f"Condition 2 is NOT fulfilled")

v=1 | LgLfh=1 | Lfh=4*x1**2 + x2
Condition 1 is fulfilled
Condition 2 is fulfilled


In [68]:
def lie_bracket(f, g, Y=None):
    if Y is None:
        dim = len(f)
        Y = []
        for i in range(dim):
            Y.append(sp.symbols(f"x{i+1}"))
    return g.jacobian(Y)*f - f.jacobian(Y)*g

def check_involutivity(f1, f2, Y=None):
    # Lie Bracket | Involutive | Dimensions
    if Y is None:
        dim = len(f1)
        Y = []
        for i in range(dim):
            Y.append(sp.symbols(f"x{i+1}"))
    F = sp.Matrix([f1.T, f2.T])
    rankF = F.rank()
    lie = lie_bracket(f1, f2, Y)
    rankFLie = sp.Matrix([f1.T, f2.T, lie.T]).rank()
    if rankF == rankFLie:
        print("It is involutive")
        return True
    else:
        print("It is NOT involutive")
        return False
    
def check_involutivity_(G, Y=None):
    D = G
    for i in range(sp.shape(G)[1]):
        f1 = G[:, i]
        for j in range(sp.shape(G)[1]):
            if not i==j:
                f1 = G[:, j]
                D = D.col_insert(sp.shape(D)[1], lie_bracket(f1, f2))
    if G.rank() == D.rank():
        print("It is involutive")
        return True
    else:
        print("It is NOT involutive")
        return False

def check_linearisability(f1, g, d=None):
    if d is None:
        d = len(f1)
        Y = []
        for i in range(d):
            Y.append(sp.symbols(f"x{i+1}"))
    G = sp.Matrix([g])
    lie = sp.Matrix([])
    for i in range(d-1):
        adf = lie_bracket(f1, G[:, i])
        G = G.col_insert(sp.shape(G)[1], adf)
    involutive = check_involutivity_(G)
    if G.rank()==len(g) and involutive:
        print("It is I/S linearisable")
        return True
    else:
        print("It is NOT I/S linearisable")
        return False

print(" ")
print("Exercise 4.6 ")
f = sp.Matrix([x2, -sp.sin(x1) - x2])
g = sp.Matrix([0, x1])
adf1 = lie_bracket(f, g)
adf2 = lie_bracket(f, adf1)
display(adf2)
        
print(" ")
print("Exercise 4.7 ")
f1 = sp.Matrix([2*x2, 1, 0])
f2 = sp.Matrix([1, 0, x2])
F = sp.Matrix([f1.T, f2.T]).T
display(F)
check_involutivity_(F)
# check_involutivity(f1, f2)

print(" ")
print("Exercise 4.8 ")
f1 = sp.Matrix([2*x3, -1, 0])
f2 = sp.Matrix([-x1, -2*x2, x3])
F = sp.Matrix([f1.T, f2.T]).T
display(F)
check_involutivity_(F)
# check_involutivity(f1, f2)

print(" ")
print("Exercise 4.9 ")
a = sp.symbols('a')
f1 = sp.Matrix([a*sp.sin(x2), -x1**2])
g = sp.Matrix([0, 1])
check_linearisability(f1, g)
# adf1 = lie_bracket(f1, g)
# G = g.col_insert(1, adf1)
# display(G)
# if G.rank()==len(f) and check_involutivity(F):
#     print("It is I/S linearisable")
# else:
#     print("It is NOT I/S linearisable")
"""
    Find flat output?
    v = n = 2
    v = 1: Lfg = dh/dx*g = dh/dx * (0, 1) = 0
           This means that h does NOT contain x2
    v = 2: Lfg2 = d(Lfg)/dx*g = d(Lfg)/dx * (0 1) =/= 0
           With Lfg = dh/dx1 * f[0], since h doesn't contain x2
           This means that h does in deed contain x1!           
"""

print(" ")
print("Exercise 4.10 ")
x1d = x1**2 + x2
x2d = x3**2 + u
x3d = x1**2 + x1 + x2
Xd = sp.Matrix([x1d, x2d, x3d])
f = sp.Matrix([
    [x1**2 + x2],
    [x3**2],
    [x1**2 + x1 + x2]
])
g = sp.Matrix([0, 1, 0])
check_linearisability(f, g)

 
Exercise 4.6 


⎡          -x₁ - 2⋅x₂           ⎤
⎢                               ⎥
⎣-x₁⋅cos(x₁) + x₁ + x₂ - sin(x₁)⎦

 
Exercise 4.7 


⎡2⋅x₂  1 ⎤
⎢        ⎥
⎢ 1    0 ⎥
⎢        ⎥
⎣ 0    x₂⎦

It is NOT involutive
 
Exercise 4.8 


⎡2⋅x₃   -x₁ ⎤
⎢           ⎥
⎢ -1   -2⋅x₂⎥
⎢           ⎥
⎣ 0     x₃  ⎦

It is involutive
 
Exercise 4.9 


ShapeError: Matrix size mismatch: (2, 2) * (3, 1).

In [58]:
G = sp.Matrix([g])
g = sp.Matrix([1, 2, 3])
for i in range(3):
    G = G.col_insert(sp.shape(G)[1], i*0.7*g)
display(G)
for i in range(sp.shape(G)[1]):
    f1 = G[:, i]
    for j in range(sp.shape(G)[1]):
        if not i==j:
            f1 = G[:, j]
            display(lie_bracket(f1, f2))

⎡1  0  0.7  1.4⎤
⎢              ⎥
⎢2  0  1.4  2.8⎥
⎢              ⎥
⎣3  0  2.1  4.2⎦

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

⎡-0.7⎤
⎢    ⎥
⎢-2.8⎥
⎢    ⎥
⎣2.1 ⎦

⎡-1.4⎤
⎢    ⎥
⎢-5.6⎥
⎢    ⎥
⎣4.2 ⎦

⎡-1⎤
⎢  ⎥
⎢-4⎥
⎢  ⎥
⎣3 ⎦

...

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

⎡-1.4⎤
⎢    ⎥
⎢-5.6⎥
⎢    ⎥
⎣4.2 ⎦

⎡-1⎤
⎢  ⎥
⎢-4⎥
⎢  ⎥
⎣3 ⎦

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

⎡-0.7⎤
⎢    ⎥
⎢-2.8⎥
⎢    ⎥
⎣2.1 ⎦

[1  cos(x₁ - x₂)]